这是一个关于pytorch的，不严谨教学，没有关于线性代数和反向传播的部分

In [ ]:

%pip install torchvision
%pip install torch
%pip install matplotlib
#先安装好依赖库

In [ ]:
# 导入所需的库
import torch
import torch.nn as nn
import torch.optim as optim




In [ ]:
import torch
import torch
import torch.nn as nn

# Define the input size and output size of the linear layer
input_size = 10
output_size = 5
linear_layer = nn.Linear(input_size, output_size)


他只能传入TORCH.TENSOR的对象，所以在与其他对象交互的时候，比如游戏场景就需要把所需的信息转成TORCH.TENSOR

In [ ]:
import torch

# Python数组
python_array = [1, 2, 3, 4, 5]

# 转换为PyTorch张量
torch_tensor = torch.tensor(python_array)

print(torch_tensor)

In [ ]:
import torch

# 二维Python数组
python_2d_array = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

# 转换为PyTorch张量
torch_2d_tensor = torch.tensor(python_2d_array)

print(torch_2d_tensor)


pytorch可以创建随机 tensor，可以用来测试或者初始化神经网络，生成噪声(总之有用)数据增强、模拟实验、概率模型。。。。。下面是示例一个维度为（1， 刚刚定义的inputsize）的随机 tensor


In [ ]:
input = torch.randn(1, input_size)
print(input)
print(input.shape)#shape 可以看到矩阵的维度，这个矩阵是1行10列的
#第一个维度在pytorch中是batch size，一般在训练的时候是多个样本一起训练，所以第一个维度是batch size
#我们不需要管理第一个维度，pytorch会自动管理这个维度

In [ ]:


output = linear_layer(input)
print(output.size())
print(output)
#input是1行10列的矩阵，linear_layer是10行5列的矩阵
#output的size是1行5列，这个矩阵是由input矩阵通过linear_layer得到的


In [ ]:
#这个linear xw+b的过程就是神经网络的前向传播过程
#w的维度是10行5列，b的维度是1行5列
#现在我们提取w和b
w = linear_layer.weight
b = linear_layer.bias
print('shape of w:', w.size())
print('shape of b:', b.size())

现在是pytorch写一个神经网络的基本方法

In [ ]:
import torch
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear_1 = nn.Linear(10, 5)
        self.linear_2 = nn.Linear(5, 2)
    def forward(self, x):
        x = self.linear_1(x)
        x = self.linear_2(x)
        return x
mlp = MLP()
input = torch.randn(1, 10)
output = mlp(input)
print(output.size())
#这里相当于定义了一个两层的神经网络，输入是10维，第一层是10维到5维，第二层是5维到2维
#这个神经网络的前向传播过程是先通过第一层，再通过第二层

forward向前计算就是推理，用户可以自己定义这个步骤，通过这种办法来达到很多操作比如 决定神经网络的计算顺序


In [ ]:
#现在我们来看看这个神经网络的参数直接打印出来是什么样的
print(mlp)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
import torchvision
import torchvision.transforms as transforms




trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                      download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                        download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=4,
                                          shuffle=False, num_workers=2)
# 这是一个nmist数据集
#
#trainset是训练集，testset是测试集
print(len(trainset))
print(len(testset))
#trainset和testset都是一个数据集，里面有60000个样本,被分成了两个数据集，一个是训练集，一个是测试集，确保训练集和测试集是独立的
# print(trainset)

In [ ]:
print(trainset[0])
#trainset[0]是一个tuple，第一个元素是一个图片，第二个元素是一个标签

In [ ]:
classes = ('0', '1', '2', '3',
              '4', '5', '6', '7', '8', '9')

import matplotlib.pyplot as plt
import numpy as np
# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)
# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))
#这里我们展示了4张图片，每张图片对应一个标签

现在开始尝试训练 
我们需要重构模型
因为这个数据集是28x28
的灰度图片

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义神经网络模型
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear_1 = nn.Linear(784, 128)  # 输入层为784，因为mnist数据集的图片大小是28*28=784
        self.relu = nn.ReLU()                # 增加非线性激活函数
        self.linear_2 = nn.Linear(128, 10)   # 输出层为10t因为mnist数据集有10个类别

    def forward(self, x):
        x = x.view(-1, 784)  # 将输入的二维图像张量展平为一维
        x = self.relu(self.linear_1(x))
        x = self.linear_2(x)
        return x

mlp = MLP()

# 将模型移动到GPU上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mlp.to(device)



In [ ]:
def train(model, device, train_loader, optimizer, loss_function):
    model.train()  
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()
        total += target.size(0)
    
    average_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    return average_loss, accuracy



下面是在测试集里测试准确率

In [ ]:
def test(model, device, test_loader, loss_function):
    model.eval() # 将模型设置为评估模式
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = loss_function(output, target)
            test_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            total += target.size(0)

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / total
    return test_loss, accuracy


In [ ]:
#写一个循环来做训练和测试
# train(mlp, trainloader, criterion, optimizer)
# test(mlp, testloader, criterion）

#现在我们把训练和测试的代码封装到一个函数里面

def train_test_loop(model, train_loader, test_loader, optimizer, loss_function, num_epochs, device, save_path='model_checkpoint/', scheduler=None, checkpoint_filename=None, save_frequency=1):
    start_epoch = 0
    for epoch in range(start_epoch, num_epochs):
        train_loss, train_accuracy = train(model, device, train_loader, optimizer, loss_function)
        test_loss, test_accuracy = test(model, device, test_loader, loss_function)

        if scheduler is not None:
            scheduler.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.01, momentum=0.9)

loss_function = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP().to(device)
num_epochs=4
# train_test_loop(model,train_loader, test_loader, 
#                 optimizer, 
#                 loss_function, 
#                 num_epochs,device,
#                 )

两层linear一般效果不会很好
换成cnn试试


cnn在图像数据上效果会更好
在pytorch中，cnn的每层的组件是torch.nn.Conv2d
这个组件的每一个参数是（输入的通道数，输出的通道数，卷积核的大小，卷积核的步长）
比如torch.nn.Conv2d(3, 16, 3, 1)表示输入的通道数是3，输出的通道数是16，卷积核的大小是3*3，卷积核的步长是1
卷积核可以被看作是一个滤波器，它会在输入的图像上滑动，每次滑动的距离是步长，然后计算卷积核和图像的内积

In [ ]:

#下面是卷积的示例 
conv = nn.Conv2d(3, 16, 3, 1)
input = torch.randn(1, 3, 32, 32)#输入的通道数是3，输入的图像大小是32*32,第一个维度是batch size不用管
output = conv(input)
print(output.size())

但是为什么输出的高宽不是一致的？

根据这些参数，可以计算输出张量的大小。输出张量的尺寸由以下公式确定：

$$
\text{output\_size} = \left\lfloor \frac{\text{input\_size} - \text{kernel\_size} + 2 \times \text{padding}}{\text{stride}} + 1 \right\rfloor
$$

其中，$\text{input\_size}$ 是输入张量的大小，$\text{kernel\_size}$ 是卷积核大小，$\text{padding}$ 是填充大小，$\text{stride}$ 是步长。

根据给定的参数：

- $\text{input\_size} = 32$ （输入图像大小）
- $\text{kernel\_size} = 3$ （卷积核大小）
- $\text{padding} = 0$ （默认情况下没有填充）
- $\text{stride} = 1$ （步长）

代入公式得到：

$$
\text{output\_size} = \left\lfloor \frac{32 - 3 + 2 \times 0}{1} + 1 \right\rfloor = \left\lfloor \frac{29}{1} + 1 \right\rfloor = 30
$$

因此，输出张量的大小为 `torch.Size([1, 16, 30, 30])`。


In [ ]:

#现在加上填充这样输出的图像大小和输入的图像大小是一样的
conv = nn.Conv2d(3, 16, 3, 1, padding=1)
input = torch.randn(1, 3, 32, 32)
output = conv(input)
print(output.size())


对于一个3x3的图像，我们假设一个2x2的卷积核，步长是1，那么输出的图像大小是2x2。

输入图像：

|   |   |   |
|---|---|---|
| 1 | 2 | 3 |
| 4 | 5 | 6 |
| 7 | 8 | 9 |

卷积核：

|   |   |
|---|---|
| 1 | 2 |
| 3 | 4 |

输出图像（计算过程）：

|   |   |
|---|---|
| 1*1+2*2+4*3+5*4=44 | 2*1+3*2+5*3+6*4=56 |
| 4*1+5*2+7*3+8*4=68 | 5*1+6*2+8*3+9*4=80 |

输出图像：

|    |    |
|----|----|
| 44 | 56 |
| 68 | 80 |

我们可以看到输出的图像是2x2的。

如果我们加上padding=1，那么输入图像变成了：

|   |   |   |   |   |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 2 | 3 | 0 |
| 0 | 4 | 5 | 6 | 0 |
| 0 | 7 | 8 | 9 | 0 |
| 0 | 0 | 0 | 0 | 0 |

卷积核仍然是：

|   |   |
|---|---|
| 1 | 2 |
| 3 | 4 |

输出图像


|   |   |   |   |
|---|---|---|---|
| 4 | 11 | 18 | 11 |
| 18 | 44 | 56 | 37 |
| 32 | 68 | 80 | 53 |
| 18 | 44 | 56 | 37 |


我们可以看到，加上padding后，输出的图像变成了4x4。



实际上卷积层的卷积核的数字是可以通过训练来学习的
他其实可以被看作一个线性层，我们下面演示一下他们的等价性

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 设置随机种子
torch.manual_seed(0)

# 定义卷积层
in_channels = 1
out_channels = 3
kernel_size = 2

#我们用相同的权重和偏置来定义一个卷积层和一个等效的线性层
# 定义卷积层
conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=0)
same_weights = torch.randn(out_channels, in_channels, kernel_size, kernel_size)
same_bias = torch.randn(out_channels)

conv_layer.weight = nn.Parameter(same_weights)
conv_layer.bias = nn.Parameter(same_bias)

# 定义等效的线性层
# 展开卷积核为线性层的权重
linear_layer = nn.Linear(in_features=in_channels*kernel_size*kernel_size, out_features=out_channels, bias=True)
#我们用相同的权重和偏置来定义一个卷积层和一个等效的线性层
linear_layer.weight = nn.Parameter(same_weights.view(out_channels, -1))#view是reshape的意思,-1表示自动计算,这里是把卷积核展平


linear_layer.bias = nn.Parameter(same_bias)




print(f"""原始权重的维度是{same_weights.shape}\n""",
      same_weights)

print(f"""扁平化:我们减少了一个维度\n
      现在是一个二维的张量{same_weights.view(out_channels, -1).shape}\n
      """,same_weights.view(out_channels, -1))



In [ ]:
#一个假设的输入图像
image = torch.randn(1, in_channels, 3, 3)  # (N, C, H, W)
print("Image:\n", image)


In [ ]:
# 卷积操作
torch_conv_result = conv_layer(image)
print("Convolution Result:\n", torch_conv_result)




In [ ]:
# 手动卷积
H, W = image.shape[2], image.shape[3]
output_height = H - kernel_size + 1
output_width = W - kernel_size + 1

# 初始化输出张量
conv_result_manual = torch.zeros((1, out_channels, output_height, output_width))

# 循环遍历每个卷积区域
for i in range(output_height):
    for j in range(output_width):
        region = image[:, :, i:i+kernel_size, j:j+kernel_size]  # 提取当前2x2区域
        region = region.reshape(1, -1)  # 使用.reshape()代替.view()
        for k in range(out_channels):
            # 计算卷积：点积加偏置
            conv_result_manual[:, k, i, j] = (linear_layer.weight[k] * region).sum() + linear_layer.bias[k]



#这里的 i是图片的x轴，j是图片的y轴
#每次取一个2*2的区域，然后把这个区域展平，然后和权重做点积，再加上偏置，就是卷积的结果
#这个过程是手动实现的，和pytorch的conv2d是一样的
#region是一个2*2的区域，展平之后是一个1*4的向量
#

print("Manual Convolution Result:\n", conv_result_manual)



In [ ]:
#我们可以拿nmist的第一个图片试一下
image = trainset[0][0].unsqueeze(0)
print(image.size())
torch_conv_result = conv_layer(image)
print(torch_conv_result.size())


In [ ]:
import torch
import matplotlib.pyplot as plt
def my_conv2d(image, kernel, bias, kernel_size, out_channels):
    # image dimensions and kernel dimensions are assumed to be correctly provided
    _, _, H, W = image.shape  # Image dimensions (N, C, H, W)
    output_height = H - kernel_size + 1
    output_width = W - kernel_size + 1

    # Initialize the output tensor
    conv_result_manual = torch.zeros((1, out_channels, output_height, output_width))

    # Loop over every pixel in the output feature map
    for i in range(output_height):
        for j in range(output_width):
            # Extract the current region to apply the kernel on
            region = image[:, :, i:i+kernel_size, j:j+kernel_size]
            region = region.reshape(1, -1)  # Flatten the region

            # Compute the convolution for each output channel
            for k in range(out_channels):
                # Perform element-wise multiplication followed by sum, then add bias
                conv_result_manual[:, k, i, j] = (kernel[k].reshape(1, -1) * region).sum() + bias[k]

    return conv_result_manual

# Parameters
in_channels = 1
out_channels = 3
kernel_size = 2
stride = 1
padding = 0

# Define the weights for the convolution kernel and the biases
same_weights = torch.randn(out_channels, in_channels, kernel_size, kernel_size)
same_bias = torch.randn(out_channels)

image = trainset[0][0].unsqueeze(0)
# Perform convolution
manual_convolution_result = my_conv2d(image, same_weights, same_bias, kernel_size, out_channels)

# Plotting the original image and the feature maps
fig, axs = plt.subplots(1, out_channels + 1, figsize=(15, 5))  # +1 for the original image

# Plot original image
axs[0].imshow(image[0, 0, :, :], cmap='gray')
axs[0].title.set_text('Original Image')
axs[0].axis('off')

# Plot each feature map
for i in range(out_channels):
    axs[i+1].imshow(manual_convolution_result[0, i, :, :], cmap='gray')
    axs[i+1].title.set_text(f'Feature Map {i+1}')
    axs[i+1].axis('off')

plt.show()

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# Parameters
in_channels = 1
out_channels = 3
kernel_size = 2

# Convolution layer setup
conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=0)


conv_layer.weight = nn.Parameter(same_weights)
conv_layer.bias = nn.Parameter(same_bias)

# Convolution operation
torch_conv_result = conv_layer(image)

# Detach the result for visualization
torch_conv_result = torch_conv_result.detach()

# Setting up the plot
fig, axs = plt.subplots(1, out_channels + 1, figsize=(15, 5))  # +1 for the original image

# Plot original image
axs[0].imshow(image[0, 0, :, :].numpy(), cmap='gray')  # Convert tensor to numpy array
axs[0].title.set_text('Original Image')
axs[0].axis('off')

# Plot each feature map
for i in range(out_channels):
    axs[i+1].imshow(torch_conv_result[0, i, :, :].numpy(), cmap='gray')  # Detach and convert tensor
    axs[i+1].title.set_text(f'Feature Map {i+1}')
    axs[i+1].axis('off')

plt.show()


可以看到他们得到的效果都是一样的

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)  # input channels = 1, output channels = 32, kernel size = 3, stride = 1
        self.conv2 = nn.Conv2d(32, 64, 3, 1)  # input channels = 32, output channels = 64, kernel size = 3, stride = 1
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(12*12*64, 128)  # considering the output dimension of the last conv layer with pooling
        self.fc2 = nn.Linear(128, 10)  # 10 classes for MNIST digits

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # should refer to 'model.parameters()' not 'mlp.parameters()'
criterion = nn.CrossEntropyLoss()

num_epochs=4
train_test_loop(model,train_loader, test_loader, 
                optimizer, 
                loss_function, 
                num_epochs,device,
                )



In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = my_conv2d(1, 32, kernel_size=3, stride=1)
        self.conv2 = my_conv2d(32, 64, kernel_size=3, stride=1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(12*12*64, 128)  # assuming input images are 28x28 and pooled once
        self.fc2 = nn.Linear(128, 10)  # 10 classes for MNIST digits

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

class my_conv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(my_conv2d, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.weights = nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size))
        self.bias = nn.Parameter(torch.randn(out_channels))

    def forward(self, x):
        batch_size, _, height, width = x.shape
        height_out = int((height - self.kernel_size) / self.stride + 1)
        width_out = int((width - self.kernel_size) / self.stride + 1)

        output = torch.zeros((batch_size, self.out_channels, height_out, width_out)).to(x.device)

        for i in range(height_out):
            for j in range(width_out):
                h_start = i * self.stride
                h_end = h_start + self.kernel_size
                w_start = j * self.stride
                w_end = w_start + self.kernel_size

                region = x[:, :, h_start:h_end, w_start:w_end]
                for k in range(self.out_channels):
                    output[:, k, i, j] = torch.sum(region * self.weights[k].unsqueeze(0), dim=[1, 2, 3]) + self.bias[k]

        return output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # should refer to 'model.parameters()' not 'mlp.parameters()'
criterion = nn.CrossEntropyLoss()

num_epochs=4
train_test_loop(model,train_loader, test_loader, 
                optimizer, 
                loss_function, 
                num_epochs,device,
                )
